# TODO
- vP is not allow to be 0 or 1
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime
- t must not be prime --> optimise
- when there is no R large/ small en
- when delta of bounds is small then the possibility of a new generation of p is high 
- when p is smaller than there is is higher chance of success
- Problem when m>=n 
- Use secret module instead of the random module for cryptographic purposes
- camelCase to snake_case if required
- gen_prime() is not used

### Import and Configuration

In [260]:
import sympy
import random
import coloredlogs, logging

In [261]:
coloredlogs.install(fmt='%(levelname)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [262]:
MIN_PRIME = 0
MAX_PRIME = 1024
OPT_RANGE_REDUCTION = 24

MAX_ITERS = 10

msg = 12345678900987654321

In [263]:
def gen_prime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def gen_prime_with_optimized_bit_limit():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def gen_prime_with_boundaries(lb, ub):
    return sympy.randprime(lb, ub)

def gen_prime_half_size():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def gen_random_number_with_bit_limit():
    return random.randint(0, pow(2, MAX_PRIME))

def fermat_prime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def modular_multiplicative_inverse(E, phiN):
    return pow(E, -1, phiN)

def calc_n(P, Q):
    return P*Q

def calc_phi_n(P, Q):
    return (P-1) * (Q-1)

def to_binary(dec):
    return "{0:b}".format(dec)

def to_decimal(bin):
    return int(bin, 2)

def concatenate_in_binary(prev, tail):
    prev = to_binary(prev)
    tail = to_binary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return to_decimal(prev + tail)

def split_in_binary(bin):
    bin = to_binary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(to_decimal(prev), to_decimal(tail))



# Voraussetzungen

In [264]:
def create_attacker_key():
    a_P = gen_prime_half_size()
    a_Q= gen_prime_half_size()


    a_PhiN = calc_phi_n(a_P, a_Q)
    a_N = calc_n(a_P, a_Q)

    a_E = fermat_prime()
    a_D = modular_multiplicative_inverse(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

Attacker = {}

created = False
while not created:
    try:
        Attacker = create_attacker_key()
        created = True
    except ValueError:
            logger.error("Trying to generate Key failed. Trying again ...")
logger.info("Attacker Key successfully generated!")

Trying to generate Key failed. Trying again ...
Trying to generate Key failed. Trying again ...


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [265]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
User = {}

valid = False
while not valid:
    # Schritt 1
    P = gen_prime_with_optimized_bit_limit()
    vP = pow(P, Attacker["E"] ,Attacker["N"])

    # Schritt 2
    t = gen_random_number_with_bit_limit() 
    tempN = concatenate_in_binary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(gen_prime_with_boundaries(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    User["P"] = P
    User["Q"] = Q

Attacker["vP"] = vP
Attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

INFO Bounds: [25094756181607148993467269918891007159791574195063344226207475799444146799910743632536552817087024845247469111859088322532837011749138659250303344810866794529046352662141491230463977507010014350239076817978896675353453385480888658488724024662475373002332547685531094249569606224359644215700451783169404929984998740, 25094756181607148993467269918891007159791574195063344226207475799444146799910743632536552817087024845247469111859088322532837011749138659250303344810866794529046352662141491230463977507010014350239076817978896675353453385480888658488724024662475373002332547685531094249569606224359644215700451783169404930023856372] 	 P: 4626357882245874545121553960813715169235091179650420271807783859638892032166004952419223427684936329394993824525362791140963557285976663788529227440816336342444746780802258825868044274831719260976837603085612696571680552405953994744252834136939412818452092390645735044806920241327911305201213897974367 	 deltaB: 38857632
INFO Amount of failed attemp

## Schritt 4

In [266]:
User["N"] = calc_n(User["P"], User["Q"])

## Schritt 5

In [267]:
def create_user_key():
    PhiN = calc_phi_n(P, Q)
    E = fermat_prime()
    D = modular_multiplicative_inverse(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        User.update(create_user_key())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO User Key successfully generated!


## Schritt 6

In [268]:
cipher = pow(msg, User["E"], User["N"])
signature = pow(msg, User["D"], User["N"])

# Angriff

## Schritt 1

In [269]:
pub_User = {
    "N" : User["N"],
    "E" : User["E"]
}

collection_User = {
    "N" : pub_User["N"],
    "E" : pub_User["E"]
}

## Schritt 2

In [270]:
collection_User["vP"] = split_in_binary(User["N"])[0]
collection_User["vP'"] = collection_User["vP"]+1

### Validation test for vP

In [271]:
testVar = "vP"
altTestVar = "vP'"

if(collection_User[testVar] == Attacker[testVar] or collection_User[altTestVar] == Attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}".format(testVar, altTestVar, Attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(Attacker[testVar], collection_User[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 645812796479908868320722351125793511614409554791119768692906525587818990843485076368235706723721442879644875786132934545307792092426303294763280102164519655306485871938125593174500946830799967713425125458277209323078943130951351596845104383809835701408206390833735841917185810205587984538778192744248578145
INFO Current vP: 645812796479908868320722351125793511614409554791119768692906525587818990843485076368235706723721442879644875786132934545307792092426303294763280102164519655306485871938125593174500946830799967713425125458277209323078943130951351596845104383809835701408206390833735841917185810205587984538778192744248578145 	 Current vP': 64581279647990886832072235112579351161440955479111976869290652558781899084348507636823570672372144287964487578613293454530779209242630329476328010216451965530648587193812559317450094683079996771342512545827720932307894313095135159684510438380983570140820639083373584191718581020558798

## Schritt 3

In [272]:
collection_User["P"] = pow(collection_User["vP"], Attacker["D"], Attacker["N"])
collection_User["P'"] = pow(collection_User["vP'"], Attacker["D"], Attacker["N"])

### Validation test for P and P'

In [273]:
testVar = "P"
altTestVar = "P'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state: {2}".format(testVar, altTestVar, User["P"]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} or {1} doesn't have the expected value.".format(testVar, altTestVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(User[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables P or P' are in the expected state: 4626357882245874545121553960813715169235091179650420271807783859638892032166004952419223427684936329394993824525362791140963557285976663788529227440816336342444746780802258825868044274831719260976837603085612696571680552405953994744252834136939412818452092390645735044806920241327911305201213897974367
INFO Current P: 4626357882245874545121553960813715169235091179650420271807783859638892032166004952419223427684936329394993824525362791140963557285976663788529227440816336342444746780802258825868044274831719260976837603085612696571680552405953994744252834136939412818452092390645735044806920241327911305201213897974367 	 Current P': 69076429577041727719571008551430855443933727813176084409960799203179992160195639042450037170689321112827054658071996102738697061260990126579429799595846703979851215395856565077933331797350805776748862268294507258602286048745674293507857301045234002906828860407095000108094912906707489895100370609

# Schritt 4

In [274]:
# Potential to optimize when N/P not hole Number
collection_User["Q"] = int(pub_User["N"]//collection_User["P"])
collection_User["Q'"] = int(pub_User["N"]//collection_User["P'"])

### Validation test for Q and Q'

In [275]:
testVar = "Q"
altTestVar = "Q'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables Q or Q' are in the expected state!
INFO Current Q: 25094756181607148993467269918891007159791574195063344226207475799444146799910743632536552817087024845247469111859088322532837011749138659250303344810866794529046352662141491230463977507010014350239076817978896675353453385480888658488724024662475373002332547685531094249569606224359644215700451783169404930018338623 	 Current Q': 1680708220947232858934947236729289049593781160927034646585091717446777604066015997767341298540644922339363233742970823362654584017989707418766038272922012610934982262661571207001410340939557710345948453821863622010883150899350456314069699259578176046578712222244019186339568711992085819979494725899815762803


## Schritt 5

In [276]:
collection_User["PhiN"] = int(calc_phi_n(collection_User["P"], collection_User["Q"]))
collection_User["PhiN'"] = int(calc_phi_n(collection_User["P'"], collection_User["Q'"]))

collection_User["D"] = modular_multiplicative_inverse(pub_User["E"], collection_User["PhiN"])
collection_User["D'"] = modular_multiplicative_inverse(pub_User["E"], collection_User["PhiN'"])

### Validation test for PhiN and PhiN'

In [277]:
testVar = "PhiN"
altTestVar = "PhiN'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables PhiN or PhiN' are in the expected state!
INFO Current PhiN: 116097323063816618934564632835327696534015468602155784725212988293620840282246091931425400039345975946130127130925884749379314510067191841411397654200388617218433608625191466215329753533320882671571502754484802318194907226845686022052469595508166449093022826141914642077325198533855930419365397321791880714101768109526418659167230392921600983726069642382203289075793492859039649964327421328027033632342607305592058107166996519599284886715056183506616084511298274972806443035333815098791982231084683793769825524698784454421303416645359321990119869347462234649343880913467274728717493786896372771564309640478463763652 	 Current PhiN': 11609732306381661893456463283532769653401546860215578472521298829362084028224609193142540003934597594613012713092588474937931451006719184141139765420038861721843360862519146621532975353332088267157150275448480231819490722684568602205246959550816644909302282614191464207732

### Validation test for D and D'

In [278]:
testVar = "D"
altTestVar = "D'"

if(collection_User[testVar] == User[testVar] or collection_User[altTestVar] == User[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_User[testVar], altTestVar, collection_User[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(Attacker[testVar], collection_User[testVar], collection_User[altTestVar]))

INFO Success!!!
INFO The variables D or D' are in the expected state!
INFO Current D: 10625322241432657588377842558345599032775756910992727722993538059190042266397791467486908913071961849411607222352159188348217471525749067927820362999434379450938626799119557110632892285165000751698824687144663995979874781369614488918789578922715143532049848348249142059901987286663531602840435984804121344022192122326921572816653919110483584851137002227878226465608883076254937218457296994453608613863786237071290485173072388491333304095654469821210663821944352858490517498907979049430921607977874077162998207076053361576193263241205200318854066807239856621858867475151085857192845686159031446112008929667055642681 	 Current D': 931921185653630906555173663666033558381517578887286474363989757718923897146363147265894993647843776278753619783727973360177853492089165082492240474010077358745462387497970224940087636621665879534211521034710468216927763627061202765172996629521966581844858906396036372699672966304615550

## Schritt 6

In [279]:
collection_User["Sig"] = pow(msg, collection_User["D"], User["N"])
collection_User["Sig'"] = pow(msg, collection_User["D'"], User["N"])

### Validation test for signature and signature'

In [280]:
testVar = "Sig"
altTestVar = "Sig'"

if(collection_User[testVar] == signature):
    collection_User["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_User[testVar]))
elif(collection_User[altTestVar] == signature):
    collection_User["correct_key"] = "D'"
    logger.info("D' is the right privat Key: {0}".format(collection_User[altTestVar]))
else:
    logger.error("Message was incorrectly signed!")

INFO D is the right privat Key: 27376525857107021028026953241047742007299483702657364510954041889525069345491254574875401879935160979645374564548972479232028320613540717064329674016780956626357154630385946170647254693186615206355906550901810384611278901895878106022199415248272649127163916439564626906254321060077890644086263817795340723596650239159115012156023389326178885077041690631127964016347299979256052403111170347892598937177936342771296665109440133223174245828228598663160320809653594205821397250049431759187061826048171981369911927840057326166875596496752842063241277033327464367122391727251065485492689686578202113212009052274072189804


# Result

In [281]:
def sort_dic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [282]:
logger.info(sort_dic(Attacker))
logger.info(sort_dic(User))
logger.info(sort_dic(collection_User))



INFO {'D': 50096515237367587812309723899158242203739466842956784808040892885913131892581699639935465614519791979517487356804821065956859213806620982991592916704921281337750256986649613501950910005134389588668449964965368713160266199954259827305839278283293986418278480926386068451991164681072115473332861896871738737985, 'E': 65537, 'N': 70546752596990902306782319671224859135487944272263237413020821183611356600789162837672717720165530135190648035044104046060609000950697680704785770688893762166223291305486241665058891579597237426363320839056330132225653916448761403828983569637739870946763092900082932499090196828662130428533688682243622647287, 'P': 8745996982232260859829415806094334798464916304624413159120481925233191849854293541126112111333126701923461710247463992126899172388126488532682673878257569, 'PhiN': 705467525969909023067823196712248591354879442722632374130208211836113566007891628376727177201655301351906480350441040460606090009506976807047857706888937453540501212345356737376685530